# Table of Contents
 <p><div class="lev2 toc-item"><a href="#Add-Global-Path" data-toc-modified-id="Add-Global-Path-01"><span class="toc-item-num">0.1&nbsp;&nbsp;</span>Add Global Path</a></div><div class="lev2 toc-item"><a href="#Load-Old-Functions" data-toc-modified-id="Load-Old-Functions-02"><span class="toc-item-num">0.2&nbsp;&nbsp;</span>Load Old Functions</a></div>

- last two weeks we used all kinds of linear regression models
- including forward stepwise regression, lasso, ridge, and elastic net
- this week we move on to multiple addictive regression models
- such as gradient boosting trees
- other models like random forest, bagging are quite similar


- among them boosting tress are the most famous and powerful
- there are several packages in this filed
- the oldest one is gbm
- the package h2o includes a parallel version of gbm
- and a newer one which has the best performance on kaggle competition is xgboost
- and currently Microsoft developed an even better package called lightgbm
- these week we can try on these models to see whether they can acutally boost our performance


- first we can install lightgbm from the zip file I sent to you
- for linux users maybe you can follow the instruction online, and it would be easier

In [ ]:
# https://github.com/Laurae2/lgbdl
# https://github.com/r-dbi/RPostgres/issues/110

In [ ]:
from stock_helper import *
from stock_stats import *
from product_info import *
from imp import reload  
import stock_helper
import stock_stats
reload(stock_helper)
reload(stock_stats)


## Add Global Path

In [ ]:
HEAD_PATH = './'
DATA_PATH = HEAD_PATH + "data/stocks/"
SAVE_PATH = HEAD_PATH + "ckpt/"    # most of signal timeseries is here
TEMP_PATH = SAVE_PATH + "tmp pkl/"
OUTPUT_PATH = HEAD_PATH + 'output'


## Load Old Functions

In [ ]:

## parallel generate the distribution of a signal
def par_get_all_signal(signal_name, file_list, product, period, SAVE_PATH="e:/intern"):
    """
    Our new functions that overwrite the stocker_helper
    """
    n_files = len(file_list)
    all_signal = np.array([])
    for file in file_list:
        S = load(SAVE_PATH+"/tmp pkl/"+product+"/"+signal_name+"/"+file) ## signal

#         good = load(SAVE_PATH+"/good pkl/"+product+"/"+file) ## good singal
#         signal = S[good]
        signal = S
        moving_average(signal,period)
        chosen = (np.arange(len(signal))+1) % period==0
        all_signal = np.concatenate((all_signal, signal[chosen]), axis=0)
    save(all_signal, SAVE_PATH+"/all signal/"+product+"."+signal_name+".pkl")

    
# calc pnl given a signal
def get_signal_pnl(file, product, signal_name, thre_mat, reverse=1,  buy_tranct=1.5e-4, sell_tranct=11.5e-4,
                   max_spread=0.011,
                   HEAD_PATH=HEAD_PATH, SAVE_PATH=SAVE_PATH, atr_filter=0):
    ## load data
    data = load(DATA_PATH+product+"/"+file)
    S = load(SAVE_PATH+"/tmp pkl/"+product+"/"+signal_name+"/"+file)
    pred = S*reverse
    atr = load(SAVE_PATH+"/tmp pkl/"+product+"/"+"atr.4096"+"/"+file)
    #n_bar = len(data)
    
    ## load signal
    
    ## we don't know the signal is positive correlated or negative correlated  
    #n_thre = len(thre_mat)
    date = np.array([x[0:10] for x in data["date.time"]])
    next_date = np.append(date[1:],'1')
    end_day = date!=next_date
    count = 0;
    n_day = sum(end_day)
    n_thre = np.shape(thre_mat)[0]
    all_pnl = np.zeros((n_day, n_thre))
    result = pd.DataFrame(data=OrderedDict([("open", thre_mat["open"].values), ("close", thre_mat["close"].values),
                               ("num", 0), ("avg.ret", 0), ("ret", 0)]), 
                          index=thre_mat.index)

    cur_spread = data["ask1"]-data["bid1"]
    for thre in thre_mat.iterrows():
        count = count+1
        buy = pred>thre[1]["open"]
        sell = pred<-thre[1]["open"]
        signal = pd.Series(data=0, index=data.index)
        position = signal.copy()
        signal[buy] = 1
        signal[sell] = -1
        signal[atr<atr_filter]=0
        scratch = -thre[1]["close"]
        position_pos = pd.Series(data=np.nan, index=data.index)
        position_pos.iloc[0] = 0
        position_pos[(signal==1) & (data["next.ask"]>0) & (data["next.bid"]>0) & (cur_spread<max_spread)] = 1
        position_pos[(pred< -scratch) & (data["next.bid"]>0) & (cur_spread<max_spread)] = 0
        position_pos.ffill(inplace=True)
        pre_pos = position_pos.shift(1)
        notional_position_pos = pd.Series(data=0, index=data.index)
        notional_position_pos[position_pos==1] = 1
        notional_position_pos[(position_pos==1) & (pre_pos==1)] = np.nan
        notional_position_pos[(notional_position_pos==1)] = 1/data["next.ask"][(notional_position_pos==1)]
        notional_position_pos.ffill(inplace=True)
        position_neg = pd.Series(data=np.nan, index=data.index)
        position_neg.iloc[0] = 0
        position_neg[(signal==-1) & (data["next.ask"]>0) & (data["next.bid"]>0) & (cur_spread<max_spread)] = -1
        position_neg[(pred> scratch) & (data["next.ask"]>0) & (cur_spread<max_spread)] = 0
        position_neg.ffill(inplace=True)
        pre_neg = position_neg.shift(1)
        notional_position_neg = pd.Series(data=0, index=data.index)
        notional_position_neg[position_neg==-1] = -1
        notional_position_neg[(position_neg==-1) & (pre_neg==-1)] = np.nan
        notional_position_neg[(notional_position_neg==-1)] = -1/data["next.bid"][(notional_position_neg==-1)]
        notional_position_neg.ffill(inplace=True)
        position = position_pos + position_neg
        notional_position = notional_position_pos+notional_position_neg
        #position[n_bar-1] = 0
        position.iloc[0] = 0
        position.iloc[-2:] = 0
        notional_position.iloc[0] = 0
        notional_position.iloc[-2:] = 0
        #change_pos = position - position.shift(1)
        #notional_change_pos = notional_position-notional_position.shift(1)
        change_pos = notional_position-notional_position.shift(1)
        change_pos.iloc[0] = 0
        #notional_change_pos.iloc[0] = 0
        change_base = pd.Series(data=0, index=data.index)
        change_buy = change_pos>0
        change_sell = change_pos<0
        change_base[change_buy] = data["next.ask"][change_buy]*(1+buy_tranct)*data["adjust"]
        change_base[change_sell] = data["next.bid"][change_sell]*(1-sell_tranct)*data["adjust"]
        raw_pnl = -(change_base*change_pos).cumsum()+notional_position*data["wpr"]
        final_pnl = -sum(change_base*change_pos) ## total pnl, there is a negative sign, because selling get money and buying pay money
        turnover = sum(change_base*abs(change_pos))
        num = sum((position!=0) & (change_pos!=0)) ## number of trades
        hld_period = sum(position!=0)   ## holding period
        daily_pnl = raw_pnl[end_day].reset_index(drop=True)
        pnl = np.append(daily_pnl[0], np.diff(daily_pnl))
        all_pnl[:,thre[0]] = pnl
        if (num==0):
            result.loc[thre[0], ("num","avg.ret","ret")] = (0,0,0)
        else:
            result.loc[thre[0],("num", "avg.ret", "ret", )] = (num, final_pnl/num, final_pnl)
    return OrderedDict([("all.pnl", all_pnl), ("result", result), ("date", date[end_day])])

    
from collections import OrderedDict
def get_signal_stat(signal_name, thre_mat, product, all_dates, CORE_NUM, split_str="2018", reverse=1, 
                     atr_filter=0, HEAD_PATH="d:/intern", SAVE_PATH="d:/intern"):
    train_sample = all_dates<split_str ## training samples
    test_sample = all_dates>split_str ## testing samples
    with dask.config.set(scheduler='processes', num_workers=CORE_NUM):
        f_par = functools.partial(get_signal_pnl, product=product, signal_name=signal_name, thre_mat=thre_mat,
                                 reverse=1,
                                HEAD_PATH=HEAD_PATH, SAVE_PATH=SAVE_PATH,atr_filter=atr_filter)
        train_result = compute([delayed(f_par)(file) for file in all_dates[train_sample]])[0] ## get training result
    train_stat = get_hft_summary(train_result, thre_mat) ## get training result statistics
    with dask.config.set(scheduler='processes', num_workers=CORE_NUM):
        f_par = functools.partial(get_signal_pnl, product=product, signal_name=signal_name, thre_mat=thre_mat,
                                 reverse=1,
                                HEAD_PATH=HEAD_PATH, SAVE_PATH=SAVE_PATH,atr_filter=atr_filter)
        test_result = compute([delayed(f_par)(file) for file in all_dates[test_sample]])[0] ## get testing result
    test_stat = get_hft_summary(test_result, thre_mat) ## get testing result statistics
    return OrderedDict([("train.stat", train_stat), ("test.stat", test_stat)])

    
def evaluate_signal(signal, all_dates, product, min_pnl, min_num, 
                    CORE_NUM, HEAD_PATH="d:/intern", SIGNAL_PATH="d:/intern", period=4096, split_str="2018", 
                    atr_filter=0, save_path="signal result",reverse=0):
    signal_name = signal+"."+str(period) ## signal name, with period
    all_signal = load(SIGNAL_PATH+"/all signal/"+product+"."+signal_name+".pkl") ## get the distribution of the signal
    open_list = np.quantile(abs(all_signal), np.append(np.arange(0.991,0.999,0.001),np.arange(0.9991,0.9999,0.0001))) ## open threshold
    thre_list = []
    for cartesian in itertools.product(open_list, np.array([0.2, 0.4, 0.6, 0.8, 1.0])): ## close threshold
        thre_list.append((cartesian[0], -cartesian[0] * cartesian[1]))
    thre_list = np.array(thre_list)
    thre_mat = pd.DataFrame(data=OrderedDict([("open", thre_list[:, 0]), ("close", thre_list[:, 1])])) ## threshold matrix
    
    if reverse>=0: ## trending signal
        print("reverse=1")
        trend_signal_stat = get_signal_stat(signal_name, thre_mat, product, all_dates, CORE_NUM, split_str=split_str, reverse=1, 
                                    atr_filter=atr_filter, HEAD_PATH=HEAD_PATH, SAVE_PATH=SIGNAL_PATH)
    if reverse<=0: ## reversal signal
        print("reverse=-1")
        reverse_signal_stat = get_signal_stat(signal_name, thre_mat, product, all_dates, CORE_NUM, split_str=split_str, reverse=-1,
                                            atr_filter=atr_filter, HEAD_PATH=HEAD_PATH, SAVE_PATH=SIGNAL_PATH)
    if reverse==0: ## both trending and reversal
        stat_result = OrderedDict([("trend.signal.stat", trend_signal_stat), ("reverse.signal.stat", reverse_signal_stat)])    
        save(stat_result, HEAD_PATH+"/"+save_path+"/"+product+"."+signal_name+".pkl")
    elif reverse==1: ## just trend
        save(trend_signal_stat, HEAD_PATH+"/"+save_path+"/"+product+"."+signal_name+".trend.pkl")
    elif reverse==-1: ## just reversal
        save(reverse_signal_stat, HEAD_PATH+"/"+save_path+"/"+product+"."+signal_name+".reverse.pkl")
        
# summarize pnl over thresholds
def get_hft_summary(result, thre_mat):
    n_thre = np.shape(thre_mat)[0]
    all_pnl = np.zeros((0,n_thre))
    all_dates = np.array([])
    for i in range(len(result)):
        all_pnl =  np.concatenate((all_pnl,result[i]["all.pnl"]),axis=0)
        all_dates = np.append(all_dates, result[i]["date"])
    stat = result[0]["result"].iloc[:,2:]
    for i in range(1,len(result)):
        stat = stat+result[i]["result"].iloc[:,2:]
    stat["avg.ret"]=stat["ret"]/stat["num"]
    
    total_ret = all_pnl.sum(0)
    total_sharpe = np.zeros(n_thre)
    total_drawdown = np.zeros(n_thre)
    total_max_drawdown = np.zeros(n_thre)
    for i in range(n_thre):
        total_sharpe[i] = sharpe(all_pnl[:,i])
        total_drawdown[i] = drawdown(all_pnl[:,i])
        total_max_drawdown[i] = max_drawdown(all_pnl[:,i])
    final_result = pd.DataFrame(data=OrderedDict([("open", thre_mat["open"]), ("close", thre_mat["close"]), ("num", stat["num"]),
                                                  ("avg.ret", stat["avg.ret"]), ("total.ret",total_ret), ("sharpe", total_sharpe),
                                                  ("drawdown", total_drawdown), ("max.drawdown", total_max_drawdown),
                                                 ("mar", total_ret/total_max_drawdown)]), 
                                index=thre_mat.index)
    return OrderedDict([("final.result", final_result), ("daily.pnl", all_pnl), ("date", all_dates)])


In [ ]:
import warnings
 
warnings.filterwarnings('ignore')

In [ ]:

def get_platform():
    import sys
    platforms = {
        'linux1' : 'Linux',
        'linux2' : 'Linux',
        'darwin' : 'OS X',
        'win32' : 'Windows'
    }
    if sys.platform not in platforms:
        return sys.platform
    
    return platforms[sys.platform]

In [ ]:
if get_platform() != 'OS X':
    CORE_NUM = int(os.environ['NUMBER_OF_PROCESSORS'])
else:
    import multiprocessing
    CORE_NUM = multiprocessing.cpu_count()

In [ ]:
CORE_NUM

In [ ]:
product_list = ["600519", "000858", "000568", "600809", "002304"]

In [ ]:
period = 4096

In [ ]:
all_dates = os.listdir(DATA_PATH + product_list[0])
all_dates.sort()
all_dates = np.array(all_dates)
n_days = len(all_dates)


In [ ]:
import dask
from dask import compute, delayed
import sklearn.preprocessing

In [ ]:
all_dates_x = os.listdir(DATA_PATH + product_list[0])
all_dates_y = os.listdir(DATA_PATH + product_list[1])
all_dates = np.array(list(set(all_dates_x) & set(all_dates_y)))
all_dates.sort()
train_sample = all_dates<"2017"
test_sample = all_dates>"2017"


In [ ]:
os.makedirs(HEAD_PATH+"/all signal", exist_ok=True)
dire_signal_list = ["nr", "dbook", "range.pos", "price.osci", "ma.dif.10", "kdj.k", "kdj.j"]
range_signal_list = ["", "range", "std", "trend.index"]
all_signal_list = np.array([])
for range_signal in range_signal_list:
    for dire_signal in dire_signal_list:
        if len(range_signal)==0:
            signal_name = dire_signal
        else:
            signal_name = dire_signal+"."+range_signal
        all_signal_list = np.append(all_signal_list,signal_name)
signal_list = [signal+".4096" for signal in all_signal_list]
np.append(signal_list, ["ret.4096", "ret.4096.001", "ret.4096.002"])
n_signal = len(signal_list)

In [ ]:
from sklearn import linear_model
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import lasso_path, LassoCV
from sklearn.preprocessing import StandardScaler
y_signal = "ret."+str(period)+".002"

- let's begin with week 10
- we can load xgboost and lightgbm at first

In [ ]:
import xgboost as xgb
import lightgbm as lgb
import os

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV

In [ ]:
ori_mat = load(SAVE_PATH+"/train test mat/"+product_list[0]+".train.mat.pkl")
signal_names = ori_mat.columns[0:-3]
signal_names
col_names = ori_mat.columns

In [ ]:
train_mat = load(SAVE_PATH+"/train test mat/"+product_list[0]+".train.mat.pkl")

print(product_list[0], train_mat.shape)
for product in product_list[1:]:
    cur_mat = load(SAVE_PATH+"/train test mat/"+product+".train.mat.pkl")
    cur_mat = pd.DataFrame(data=sklearn.preprocessing.scale(cur_mat, with_mean=False), columns=col_names)
    print(product, cur_mat.shape)
    train_mat = train_mat.append(cur_mat)

# 600519 (1001, 32)
# 000858 (1281, 32)
# 000568 (1007, 32)
# 600809 (756, 32)
# 002304 (838, 32)






In [ ]:
train_std_mat = dict([])
train_mat = load(SAVE_PATH+"/train test mat/"+product_list[0]+".train.mat.pkl")
train_mat = train_mat[0:0]
test_mat = train_mat
for product in product_list:
    cur_train_mat = load(SAVE_PATH+"/train test mat/"+product+".train.mat.pkl")
    train_std_mat[product] = np.std(cur_train_mat)
    train_mat = train_mat.append(cur_train_mat/train_std_mat[product])
    cur_test_mat = load(SAVE_PATH+"/train test mat/"+product+".test.mat.pkl")
    test_mat = test_mat.append(cur_test_mat/train_std_mat[product])

In [ ]:
print(train_mat.shape)
print(test_mat.shape)

# (4883, 32)
# (4845, 32)





In [ ]:
x_train = train_mat.iloc[:, :n_signal].values
y_train = train_mat.loc[:, y_signal].values

In [ ]:
strat = "gbm.wine.4096"

- create a directory for the first strate

In [ ]:
for product in product_list:
    #os.makedirs(SAVE_PATH+"/tmp pkl/"+product+"/"+strat, exist_ok=True)
    os.makedirs(TEMP_PATH+product+"/"+strat, exist_ok=True)

    

In [ ]:
gb1 = GradientBoostingRegressor(random_state=0)
param_grid = {
        'n_estimators': np.arange(10,200,10),
        'max_depth': [3,4,5,6,7,8,9,10],
        'learning_rate':[0.01,0.1],
        'min_samples_leaf': [10,20], 
    
}


In [ ]:
%%time
gbm1 = GridSearchCV(gb1, param_grid, n_jobs=CORE_NUM, cv=5);
gbm1.fit(x_train, y_train);

# Wall time: 17min 24s

# GridSearchCV(cv=5, error_score='raise-deprecating',
#              estimator=GradientBoostingRegressor(alpha=0.9,
#                                                  criterion='friedman_mse',
#                                                  init=None, learning_rate=0.1,
#                                                  loss='ls', max_depth=3,
#                                                  max_features=None,
#                                                  max_leaf_nodes=None,
#                                                  min_impurity_decrease=0.0,
#                                                  min_impurity_split=None,
#                                                  min_samples_leaf=1,
#                                                  min_samples_split=2,
#                                                  min_weight_fraction_leaf=0.0,
#                                                  n_estimators=100,
#                                                  n_iter...
#                                                  subsample=1.0, tol=0.0001,
#                                                  validation_fraction=0.1,
#                                                  verbose=0, warm_start=False),
#              iid='warn', n_jobs=16,
#              param_grid={'learning_rate': [0.01, 0.1],
#                          'max_depth': [3, 4, 5, 6, 7, 8, 9, 10],
#                          'min_samples_leaf': [10, 20],
#                          'n_estimators': array([ 10,  20,  30,  40,  50,  60,  70,  80,  90, 100, 110, 120, 130,
#        140, 150, 160, 170, 180, 190])},
#              pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
#              scoring=None, verbose=0)



In [ ]:
print('Gradient boosted tree regression...')
print('Best Params:')
print(gbm1.best_params_)
print('Best CV Score:')
print(gbm1.best_score_)
save(gbm1, HEAD_PATH+"/wine.gbm.4096.pkl")

# Gradient boosted tree regression...
# Best Params:
# {'learning_rate': 0.01, 'max_depth': 3, 'min_samples_leaf': 20, 'n_estimators': 120}
# Best CV Score:
# 0.0016858221075438632





In [ ]:
gbm1 = load(HEAD_PATH+"/wine.gbm.4096.pkl")

In [ ]:
%%time
for product in product_list:
    parLapply(CORE_NUM, all_dates, get_daily_gbm, 
          product=product, signal_list=signal_list, 
          model=gbm1, strat=strat, HEAD_PATH=HEAD_PATH, SAVE_PATH=SAVE_PATH, train_std=np.array(train_std_mat[product][:n_signal]))
;
# Wall time: 6min 19s

In [ ]:
signal_name = strat
print(strat)

In [ ]:
%%time
for product in product_list:
    par_get_all_signal(strat, all_dates, product, 4096, SAVE_PATH=SAVE_PATH)

## Wall time:2.76 s

In [ ]:
%%time
# TODO: why all signal? use to be signal result atr
for product in product_list:
    evaluate_signal("gbm.wine", all_dates, product, 0.001, 20, CORE_NUM, HEAD_PATH, SAVE_PATH,
                        period=4096, split_str="2017",atr_filter=0.02, save_path="ckpt/all signal", reverse=1)


# reverse=1
# reverse=1
# reverse=1
# reverse=1
# reverse=1
# Wall time: 23min 22s
# Wall time: 23min 22s



In [ ]:
for product in product_list:
    signal_stat = load("ckpt/all signal/"+product+"."+"gbm.wine.4096.trend"+".pkl")
    train_stat = signal_stat["train.stat"]
    good_strat = (train_stat["final.result"]["avg.ret"]>0.001)
    train_pnl = train_stat["daily.pnl"][:, good_strat].sum(axis=1)/sum(good_strat)
    test_stat = signal_stat["test.stat"]
    test_pnl = test_stat["daily.pnl"][:, good_strat].sum(axis=1)/sum(good_strat)
    print(product, "train sharpe ", sharpe(train_pnl), "test sharpe ", sharpe(test_pnl))

# 600519 train sharpe  0.22919983414628217 test sharpe  -0.21748311895543404
# 000858 train sharpe  0.18525756029068166 test sharpe  -1.2281289356720344
# 000568 train sharpe  0.22594233445858758 test sharpe  -0.7105898416531242
# 600809 train sharpe  0.3274442411746926 test sharpe  -1.1298196939247898
# 002304 train sharpe  0.4769715509858891 test sharpe  -0.6685253299035706



In [ ]:
## construct the signal matrix
## we want to put multiple signals into a matrix
def get_sample_signal(good_night_files, sample, product, signal_list, period, daily_num):
    n_samples = sum(daily_num[sample]) ## tottal number of samples
    n_signal = len(signal_list) ## number of signals, the matrix would be n_samples*n_signal
    all_signal =  np.ndarray(shape=(int(n_samples),n_signal))
    cur = 0
    for file in good_night_files[sample]:
        data = load(DATA_PATH+product+"/"+file)
        chosen = (np.arange(np.shape(data)[0])+1) % period==0
        n_chosen = sum(chosen)
        for i in range(n_signal):
            signal_name = signal_list[i]
            #S = load(SAVE_PATH+"/tmp pkl/"+product+"/"+signal_name+"/"+file)
            S = load(TEMP_PATH + product + "/" + signal_name + "/" + file)
            signal = S[(np.arange(len(S))+1) % period == 0]
            signal[np.isnan(signal)] = 0 ## the ret.cor has some bad records
            signal[np.isinf(signal)] = 0 ## the ret.cor has some bad records
            all_signal[cur:(cur+n_chosen),i] = signal
        cur = cur+n_chosen
    all_signal = pd.DataFrame(all_signal, columns=signal_list)
    return all_signal

- now we can go on with rolling gbm model
- it's added this term of course

In [ ]:
## rolling model for gradient boosting machine
## returned value is the training sample standard deviation and the final model
def get_multiple_gbm_roll_model(train_start, train_end, y_signal, strat, 
                          product_list, all_dates, daily_num, signal_list, period=4096, SAVE_PATH=SAVE_PATH):
    scaler =  StandardScaler(copy=True, with_mean=False, with_std=True)
    gb1 = GradientBoostingRegressor(random_state=0)
    param_grid = {
            'n_estimators': np.arange(20,300,20),
            'max_depth': [3,4,5,6,7,8],
            'learning_rate':[0.01],
            'min_samples_leaf': [10], 
    }
    model = GridSearchCV(gb1, param_grid, n_jobs=CORE_NUM, cv=5);
    coef_list = dict([])
    train_std_mat = dict([])
    y_std = dict([])
    n_signal = len(signal_list)
    train_mat = np.zeros((0,n_signal))
    y_train = np.array([])
    train_start = 0
    train_end = 2
    sample  =(all_dates>=all_dates[train_start]) & (all_dates<=all_dates[train_end])
    for product in product_list: ## combine data sets together and then fit the model
        print(product)
        train = get_sample_signal(all_dates, sample, product, np.append(signal_list, y_signal), period, daily_ticks[product])
        x_train = train.iloc[:,:n_signal]
        scaler.fit(x_train)
        cur_y_train = train[y_signal]
        x_train = scaler.transform(x_train) ## normalize data before combine
        train_std_mat[product] = np.sqrt(scaler.var_)
        train_mat = np.append(train_mat,x_train, axis=0)
        y_std[product] = np.std(cur_y_train)
        y_train = np.append(y_train, cur_y_train/y_std[product])
    model.fit(train_mat, y_train);
    os.makedirs(SAVE_PATH+"/model", exist_ok=True)
    save(model, SAVE_PATH+"/model/"+strat+".pkl")
    


In [ ]:
daily_ticks = dict([])
all_mat = dict([])
for product in product_list:
    daily_num = load(SAVE_PATH+"/daily num/"+product+".pkl")
    daily_ticks[product] = daily_num
    test_mat = load(SAVE_PATH+"/train test mat/"+product+".test.mat.pkl")
    train_mat = load(SAVE_PATH+"/train test mat/"+product+".train.mat.pkl")
    all_mat[product] = pd.concat((train_mat, test_mat))

In [ ]:
%%time
strat = "gbm.wine.4096"
y_signal = "ret.4096.002"
train_end = 6
for train_start in range(4):
    print(all_dates[train_start])
    get_multiple_gbm_roll_model(train_start, train_end, y_signal, strat+"."+all_dates[train_start], 
                          product_list, all_dates, daily_num, signal_list, SAVE_PATH=SAVE_PATH)
    train_start = train_start+1
    train_end = train_end+1

# 2011.pkl
# 600519
# 000858
# 000568
# 600809
# 002304
# 2012.pkl
# 600519
# 000858
# 000568
# 600809
# 002304
# 2013.pkl
# 600519
# 000858
# 000568
# 600809
# 002304
# 2014.pkl
# 600519
# 000858
# 000568
# 600809
# 002304
# Wall time: 11min 16s


- now we can generate values of each model

In [ ]:

def get_daily_gbm(file_name, product, signal_list, model, strat, HEAD_PATH, SAVE_PATH, train_std, thre=float('Inf')):
    signal_mat = load(SAVE_PATH+"/signal mat pkl/"+product+"/"+file_name).T/train_std
    S = model.predict(signal_mat)
    S[np.abs(S)>thre] = 0
    #save(S, SAVE_PATH+"/tmp pkl/"+product+"/"+strat+"/"+file_name)
    save(S, TEMP_PATH + product + "/" + strat + "/" + file_name)
    
    

In [ ]:
%%time
start_month = 0
train_end = 6
for train_start in range(4):
    print(all_dates[train_start])
    strat_name = strat+"."+all_dates[train_start]
    gbm1 = load(SAVE_PATH+"/model/"+strat_name+".pkl")
    #gbm1 = load(SAVE_PATH+"/model/"+strat_name)

    for product in product_list:
        #os.makedirs(SAVE_PATH+"/tmp pkl/"+product+"/"+strat_name, exist_ok=True)
        os.makedirs(TEMP_PATH+product+"/"+strat_name, exist_ok=True)
        ## each model has training set from train_start to train_end, and a test set until train_end+forward_len, so 
        ## we need to generate values of all_dates[train_start:(train_end+forward_len)+1], plus 1 to include train_end+forward_len
        parLapply(CORE_NUM, all_dates, get_daily_gbm, product=product, signal_list=signal_list, 
                  model=gbm1, strat=strat_name, HEAD_PATH=HEAD_PATH, SAVE_PATH=SAVE_PATH, train_std=np.array(train_std_mat[product][:n_signal]))
    train_end = train_end+1

# 2011.pkl
# 2012.pkl
# 2013.pkl
# 2014.pkl
# Wall time: 4min 30s


In [ ]:
%%time
strat = "gbm.wine.4096"
train_end = 6
for train_start in range(4):
    print(all_dates[train_start])
    ## we only generte distribution on training set, not on test set
    for product in product_list:
        par_get_all_signal(strat+"."+all_dates[train_start], all_dates[train_start:(train_end+1)], product, 4096, SAVE_PATH=SAVE_PATH)
    train_end = train_end+1
    

# 2011.pkl
# 2012.pkl
# 2013.pkl
# 2014.pkl
# Wall time: 6.19 s



In [ ]:
## get rolling statistics of the strategy performance
def get_roll_result(product,strat,train_range, train_end,  atr_filter=0, save_path="signal result atr", 
                          HEAD_PATH=SAVE_PATH, SIGNAL_PATH=SAVE_PATH):
    for train_start in range(train_range):
        strat_name = strat+"."+all_dates[train_start]
        print(product, strat_name)
        all_signal = load(SIGNAL_PATH+"/all signal/"+product+"."+strat_name+".pkl")
        open_list = np.quantile(abs(all_signal), np.append(np.arange(0.991,0.999,0.001),np.arange(0.9991,0.9999,0.0001)))
        thre_list = []
        for cartesian in itertools.product(open_list, np.array([0.2, 0.4, 0.6, 0.8, 1.0])):
            thre_list.append((cartesian[0], -cartesian[0] * cartesian[1]))
        thre_list = np.array(thre_list)
        thre_mat = pd.DataFrame(data=OrderedDict([("open", thre_list[:, 0]), ("close", thre_list[:, 1])]))
        train_result = parLapply(CORE_NUM, all_dates[train_start:(train_end+1)], get_signal_pnl,
                                 product=product, signal_name=strat_name, thre_mat=thre_mat, reverse=1,
                                 HEAD_PATH=HEAD_PATH, SAVE_PATH=SIGNAL_PATH,atr_filter=atr_filter)
        train_stat = get_hft_summary(train_result, thre_mat)
        test_result = parLapply(CORE_NUM, all_dates[(train_end+1):(train_end+2)], get_signal_pnl, 
                            product=product, signal_name=strat_name, thre_mat=thre_mat, reverse=1,
                            HEAD_PATH=HEAD_PATH, SAVE_PATH=SIGNAL_PATH, atr_filter=atr_filter)
        test_stat = get_hft_summary(test_result, thre_mat)
        stat_result = OrderedDict([("train.stat", train_stat), ("test.stat", test_stat)])    
        save(stat_result, HEAD_PATH+"/"+save_path+"/"+product+"."+strat_name+".pkl")
        train_end = train_end+1

In [ ]:
%%time
strat = "gbm.wine.4096"
for product in product_list:
    get_roll_result(product, strat,4,6, atr_filter=0.02)

# 600519 gbm.wine.4096.2011.pkl
# 600519 gbm.wine.4096.2012.pkl
# 600519 gbm.wine.4096.2013.pkl
# 600519 gbm.wine.4096.2014.pkl
# 000858 gbm.wine.4096.2011.pkl
# 000858 gbm.wine.4096.2012.pkl
# 000858 gbm.wine.4096.2013.pkl
# 000858 gbm.wine.4096.2014.pkl
# 000568 gbm.wine.4096.2011.pkl
# 000568 gbm.wine.4096.2012.pkl
# 000568 gbm.wine.4096.2013.pkl
# 000568 gbm.wine.4096.2014.pkl
# 600809 gbm.wine.4096.2011.pkl
# 600809 gbm.wine.4096.2012.pkl
# 600809 gbm.wine.4096.2013.pkl
# 600809 gbm.wine.4096.2014.pkl
# 002304 gbm.wine.4096.2011.pkl
# 002304 gbm.wine.4096.2012.pkl
# 002304 gbm.wine.4096.2013.pkl
# 002304 gbm.wine.4096.2014.pkl
# Wall time: 1h 30min 16s



In [ ]:
# TODO: why the results different?
n_product = len(product_list)
save_path = "signal result atr"
strat = "gbm.wine.4096"
start_month = 0
min_pnl = 0.001
min_num = 10

train_end = 6
for i_product in range(n_product):
    all_pnl = np.array([])    
    for train_start in range(4):
        strat_name = strat+"."+all_dates[train_start]
        product = product_list[i_product]
        stat_result = load(SAVE_PATH+"/"+save_path+"/"+product+"."+strat_name+".pkl")
        train_stat = stat_result["train.stat"]
        test_stat = stat_result["test.stat"]
        good_strat = (train_stat["final.result"]["avg.ret"]>min_pnl) & (train_stat["final.result"]["num"]>min_num)
        train_pnl = train_stat["daily.pnl"][:, good_strat].sum(axis=1)/sum(good_strat) 
        train_pnl[np.isnan(train_pnl)]=0
        test_pnl = test_stat["daily.pnl"][:, good_strat].sum(axis=1)/sum(good_strat)
        test_pnl[np.isnan(test_pnl)]=0
        all_pnl = np.append(all_pnl, test_pnl)
    plt.figure(i_product, figsize=(16, 10))
    plt.title(product_list[i_product])
    plt.ylabel("pnl")
    plt.plot(all_pnl.cumsum());
    print(product, "sharpe: ", sharpe(all_pnl))
    
# 600519 sharpe:  0.30726873760820356
# 000858 sharpe:  0.8924871261897995
# 000568 sharpe:  0.6929074966143157
# 600809 sharpe:  1.1465550163246363
# 002304 sharpe:  -0.765025038090387


- Now let's start with xgboost

In [ ]:
import xgboost as xgb

- we need to create specific train and test data sets for xgboost

In [ ]:
cv_params = {'max_depth': [3,4,5,6,7,8,9,10], 
             'n_estimators': np.arange(10,200,10)}
ind_params = {'learning_rate': 0.1,
              "min_samples_leaf":[10,20],
              'seed':100, 'objective': 'reg:linear'}
xgb_cv = GridSearchCV(xgb.XGBRegressor(**ind_params), 
                            cv_params, 
                            cv = 5, n_jobs = -1) 

In [ ]:
%%time
xgb_cv.fit(x_train, y_train);

# Wall time: 14min 33s

# GridSearchCV(cv=5, error_score='raise-deprecating',
#              estimator=XGBRegressor(base_score=0.5, booster='gbtree',
#                                     colsample_bylevel=1, colsample_bytree=1,
#                                     gamma=0, importance_type='gain',
#                                     learning_rate=0.1, max_delta_step=0,
#                                     max_depth=3, min_child_weight=1,
#                                     min_samples_leaf=[10, 20], missing=None,
#                                     n_estimators=100, n_jobs=1, nthread=None,
#                                     objective='reg:linear', random_state=0,
#                                     reg_alpha=0, reg_lambda=1,
#                                     scale_pos_weight=1, seed=100, silent=True,
#                                     subsample=1),
#              iid='warn', n_jobs=-1,
#              param_grid={'max_depth': [3, 4, 5, 6, 7, 8, 9, 10],
#                          'n_estimators': array([ 10,  20,  30,  40,  50,  60,  70,  80,  90, 100, 110, 120, 130,
#        140, 150, 160, 170, 180, 190])},
#              pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
#              scoring=None, verbose=0)


In [ ]:
#best_parameters, score, _ = max(xgb_cv.grid_scorer, key=lambda x: x[1])
xgb_cv.best_params_
## {'max_depth': 3, 'n_estimators': 30}

In [ ]:
SAVE_PATH

In [ ]:
#save(xgb_cv, "e:/intern/xgb_crypto.pkl")
save(xgb_cv, SAVE_PATH+"xgb_crypto.pkl")

In [ ]:
xgb_cv = load( SAVE_PATH + "xgb_crypto.pkl")

In [ ]:
strat = "xgb.wine.4096"
signal_name = strat

In [ ]:
for product in product_list:
    #os.makedirs(SAVE_PATH+"/tmp pkl/"+product+"/"+strat, exist_ok=True)
    os.makedirs(TEMP_PATH+product+"/"+strat, exist_ok=True)


- we can plot them to check the result

In [ ]:
%%time
for product in product_list:
    parLapply(CORE_NUM, all_dates, get_daily_gbm, 
          product=product, signal_list=signal_list, 
          model=xgb_cv, strat=strat, HEAD_PATH=HEAD_PATH, SAVE_PATH=SAVE_PATH, train_std=np.array(train_std_mat[product][:n_signal]))
;
# Wall time: 7min 5s

In [ ]:
%%time
for product in product_list:
    par_get_all_signal(strat, all_dates, product, 4096, SAVE_PATH=SAVE_PATH)
    
#  Wall time:  773 ms


In [ ]:
%%time
for product in product_list:
    evaluate_signal("xgb.wine", all_dates, product, 0.001, 20, CORE_NUM, HEAD_PATH, SAVE_PATH,
                        period=4096, split_str="2017",atr_filter=0.02, save_path=SAVE_PATH+"signal result atr", reverse=1)

# reverse=1
# reverse=1
# reverse=1
# reverse=1
# reverse=1
# Wall time: 49min 29s



In [ ]:
for product in product_list:
    signal_stat = load(SAVE_PATH+"/signal result atr/"+product+"."+"xgb.wine.4096.trend"+".pkl")
    train_stat = signal_stat["train.stat"]
    good_strat = (train_stat["final.result"]["avg.ret"]>0.001)
    train_pnl = train_stat["daily.pnl"][:, good_strat].sum(axis=1)/sum(good_strat)
    test_stat = signal_stat["test.stat"]
    test_pnl = test_stat["daily.pnl"][:, good_strat].sum(axis=1)/sum(good_strat)
    print(product, "train sharpe ", sharpe(train_pnl), "test sharpe ", sharpe(test_pnl))

# 600519 train sharpe  0.1388950546824526 test sharpe  -0.36275758109400175
# 000858 train sharpe  0.6539396017038269 test sharpe  0.19379771958752676
# 000568 train sharpe  0.5938268485645808 test sharpe  0.008024859731152694
# 600809 train sharpe  0.3510481091807721 test sharpe  0.024465093844693718
# 002304 train sharpe  0.3935124085400787 test sharpe  -0.538542431125311




- SUMMARY


- today we compare two tree-based models: gradient boosting machine and xgboost
- but the result is pretty bad
- you can try to tune the parameter to see whether we could improve it